## 1. Import libraries 

In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## 2. Load and merge data

In [2]:
# Reading csv files

df1 = pd.read_csv('store.csv',dtype={'Store':'string', 'StoreType':'string', 'Assortment':'string'})
df2 = pd.read_csv('train.csv',dtype={'Store':'string', 'StateHoliday':'string'})

# Merging into one

df = pd.merge(df1, df2, 
                   on='Store', 
                   how='inner')

## 3. Split data (train and test)


In [3]:
# First step in order to avoid information leaks
# Train test split processing

X = df.drop(columns='Sales')
y = df.loc[:,'Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Merge train dataset to analyse data and feature engineering

full_train = [y_train, X_train]
full_test = [y_test, X_test]

df_train = pd.concat(full_train, axis=1)
df_test = pd.concat(full_test, axis=1)

## 4. Data inspection

In [4]:
# Use Profiling to get information about our training dataframe and export file

#prof_1 = ProfileReport(df_train)

#prof_1.to_file(output_file='prof_1.html')


In [5]:
df_train.head()

,Sales,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Date,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday
290482,0.0,524,a,c,40860.0,9.0,2013.0,0,NaN,NaN,NaN,2014-02-02,7.0,0.0,0.0,0.0,0,0.0
139915,6267.0,253,a,c,250.0,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov",2013-07-19,5.0,749.0,1.0,1.0,0,0.0
318657,0.0,575,a,a,960.0,5.0,2008.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2013-11-24,7.0,0.0,0.0,0.0,0,0.0
361617,5900.0,653,d,c,7520.0,7.0,2014.0,1,45.0,2009.0,"Feb,May,Aug,Nov",2013-03-01,NaN,548.0,1.0,0.0,0,0.0
374646,8468.0,676,b,b,1410.0,9.0,2008.0,0,NaN,NaN,NaN,2013-12-17,2.0,1767.0,1.0,1.0,0,0.0


## 5. Featuring engineering

### 5.1 Operations

1.Sales :Eliminate rows with 0.0 and NaN due to the fact that I am using the RMSPE as a metric.

2.Store :Transfor to String and use encoding to use as a categorical

3.StoreType : Impute Mode as value for NaNs. Encoding to use as a categorical.

4.Assortment : Use Mode as value for NaNs.Encoding to use as a categorical

5.CompetitionDistance :Impute Median

6.CompetitionOpenSinceMonth & CompetitionOpenSinceYear: dropped, with extra time they could be combine to generate another useful feature

7.Promo2. Impute 0 for NaN. (most probable)

8.Promo2SinceWeek & Promo2SinceYear create a timeseries dummy from weeks and months obtaining a competition start date and if there is promo active.

9.PromoInterval: drop. It could be also usea imputing mode and encoding.

10.Date:Tranform Data into datatime pd to get the day, week and year.

11.Day of week: impute mode

12.Customers: Drop

13.Open: Impute 1 if there is sales and Open value is NaN.

14.StateHoliday: fillna with 0 and get a string

15.SchoolHoliday: fillna with 0 and get a string

15.Feature creation: Average Sales-Store & Sales-Storetype


In [6]:
df_train.head()

,Sales,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Date,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday
290482,0.0,524,a,c,40860.0,9.0,2013.0,0,NaN,NaN,NaN,2014-02-02,7.0,0.0,0.0,0.0,0,0.0
139915,6267.0,253,a,c,250.0,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov",2013-07-19,5.0,749.0,1.0,1.0,0,0.0
318657,0.0,575,a,a,960.0,5.0,2008.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2013-11-24,7.0,0.0,0.0,0.0,0,0.0
361617,5900.0,653,d,c,7520.0,7.0,2014.0,1,45.0,2009.0,"Feb,May,Aug,Nov",2013-03-01,NaN,548.0,1.0,0.0,0,0.0
374646,8468.0,676,b,b,1410.0,9.0,2008.0,0,NaN,NaN,NaN,2013-12-17,2.0,1767.0,1.0,1.0,0,0.0


### 5.2 Data engineering

In [7]:
#Drop NaN and 0.0 values in sales

df_train = df_train.dropna(subset =['Sales'])
df_train = df_train.drop(df_train[df_train['Sales']==0].index)

df_test = df_test.dropna(subset =['Sales'])
df_test = df_test.drop(df_test[df_test['Sales']==0].index)


#Drop columns Customers and PromoInterval

df_train = df_train.drop(columns=['Customers','PromoInterval'])
df_test = df_test.drop(columns=['Customers','PromoInterval'])
df_train


,Sales,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Date,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
139915,6267.0,253,a,c,250.0,NaN,NaN,1,5.0,2013.0,2013-07-19,5.0,1.0,1.0,0,0.0
361617,5900.0,653,d,c,7520.0,7.0,2014.0,1,45.0,2009.0,2013-03-01,NaN,1.0,0.0,0,0.0
374646,8468.0,676,b,b,1410.0,9.0,2008.0,0,NaN,NaN,2013-12-17,2.0,1.0,1.0,0,0.0
166,5011.0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,2013-06-20,4.0,1.0,1.0,0,0.0
194544,5437.0,351,a,a,5290.0,11.0,2012.0,1,5.0,2013.0,2014-06-07,6.0,1.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54886,4860.0,99,c,c,2030.0,11.0,2003.0,1,22.0,2012.0,2014-04-26,NaN,1.0,0.0,0,0.0
259178,4860.0,468,c,c,5260.0,9.0,2012.0,0,NaN,NaN,2013-02-11,1.0,1.0,0.0,0,0.0
365838,5691.0,660,a,a,1200.0,11.0,2006.0,1,40.0,2014.0,2014-01-10,5.0,1.0,1.0,0,0.0
131932,4180.0,239,d,c,610.0,NaN,NaN,0,NaN,NaN,2013-02-23,6.0,1.0,0.0,0,0.0


In [8]:
#Feature creation  Average Sales-Store 

sales_avg_store_train = df_train.groupby('Store')['Sales'].mean().reset_index()
sales_avg_store_train = sales_avg_store_train.rename(columns = {'Sales':'store_encode'})

sales_avg_store_test = df_test.groupby('Store')['Sales'].mean().reset_index()
sales_avg_store_test = sales_avg_store_test.rename(columns = {'Sales':'store_encode'})

#Merging into DF
df_train = pd.merge(df_train,sales_avg_store_train,on='Store',how='left',suffixes=('', ''))
df_test = pd.merge(df_test,sales_avg_store_test,on='Store',how='left',suffixes=('', ''))



In [9]:
# Feature creation Sales-Storetype

avg_sales_storetype_train = df_train.groupby(['StoreType','Assortment'])['Sales'].mean().reset_index()
avg_sales_storetype_train = avg_sales_storetype_train.rename(columns = {'Sales':'sales_storetype'})

avg_sales_storetype_test = df_test.groupby(['StoreType','Assortment'])['Sales'].mean().reset_index()
avg_sales_storetype_test = avg_sales_storetype_test.rename(columns = {'Sales':'sales_storetype'})

#merge on the mean sales per storetype and assortment.

df_train = pd.merge(df_train,avg_sales_storetype_train,on=['StoreType','Assortment'],how='left',suffixes=('',''))
df_test = pd.merge(df_test,avg_sales_storetype_test,on=['StoreType','Assortment'],how='left',suffixes=('', ''))

df_train

,Sales,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Date,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,store_encode,sales_storetype
0,6267.0,253,a,c,250.0,NaN,NaN,1,5.0,2013.0,2013-07-19,5.0,1.0,1.0,0,0.0,5908.065574,7425.648995
1,5900.0,653,d,c,7520.0,7.0,2014.0,1,45.0,2009.0,2013-03-01,NaN,1.0,0.0,0,0.0,5854.905149,6843.009539
2,8468.0,676,b,b,1410.0,9.0,2008.0,0,NaN,NaN,2013-12-17,2.0,1.0,1.0,0,0.0,7000.188544,8152.210169
3,5011.0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,2013-06-20,4.0,1.0,1.0,0,0.0,4782.823684,6762.479180
4,5437.0,351,a,a,5290.0,11.0,2012.0,1,5.0,2013.0,2014-06-07,6.0,1.0,0.0,0,0.0,5882.175141,6491.087624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397974,4860.0,99,c,c,2030.0,11.0,2003.0,1,22.0,2012.0,2014-04-26,NaN,1.0,0.0,0,0.0,4825.462462,6853.789405
397975,4860.0,468,c,c,5260.0,9.0,2012.0,0,NaN,NaN,2013-02-11,1.0,1.0,0.0,0,0.0,6979.093220,6853.789405
397976,5691.0,660,a,a,1200.0,11.0,2006.0,1,40.0,2014.0,2014-01-10,5.0,1.0,1.0,0,0.0,4482.273273,6491.087624
397977,4180.0,239,d,c,610.0,NaN,NaN,0,NaN,NaN,2013-02-23,6.0,1.0,0.0,0,0.0,4857.355932,6843.009539


In [10]:
#Dealing with missing values CompetitionDistance

df_train['CompetitionDistance']=df_train.loc[:,'CompetitionDistance'].fillna(df_train['CompetitionDistance'].median())
df_test['CompetitionDistance']=df_test.loc[:,'CompetitionDistance'].fillna(df_test['CompetitionDistance'].median())

#Open: filling NaN using Sales

df_train.loc[df_train['Sales']> 0, 'Open']=1
df_train.loc[df_train['Sales']== 0, 'Open']=0

df_test.loc[df_test['Sales']> 0, 'Open']=1
df_test.loc[df_test['Sales']== 0, 'Open']=0

#SatateHoloiday: fillna with 0 and get a string

df_train['StateHoliday'] = df_train.loc[:,'StateHoliday'].fillna('0')
df_train['StateHoliday'] = df_train['StateHoliday'].replace(0.0, '0')

df_test['StateHoliday'] = df_test.loc[:,'StateHoliday'].fillna('0')
df_test['StateHoliday'] = df_test['StateHoliday'].replace(0.0, '0')

#Schoolholiday fillna using 0

df_train['SchoolHoliday'] = df_train['SchoolHoliday'].fillna(0)
df_test['SchoolHoliday'] = df_test['SchoolHoliday'].fillna(0)

#Promo missing values (0)
df_train['Promo'] = df_train.loc[:,'Promo'].fillna(0)
df_test['Promo'] = df_test.loc[:,'Promo'].fillna(0)

#DayOfWeek: filling NaN using Mode (2)

df_train['DayOfWeek'] = df_train['DayOfWeek'].fillna(1)
df_test['DayOfWeek'] = df_test['DayOfWeek'].fillna(1)




In [11]:
#Tranform Data into datatime pd to get the day, week and year

df_train['Sales_date'] = pd.to_datetime(df_train['Date'])
df_train['Year'] = pd.DatetimeIndex(df_train['Date']).year
df_train['Month'] = pd.DatetimeIndex(df_train['Date']).month
df_train['Day'] = pd.DatetimeIndex(df_train['Date']).day

df_test['Sales_date'] = pd.to_datetime(df_train['Date'])
df_test['Date'] = pd.to_datetime(df_test['Date'])
df_test['Year'] = pd.DatetimeIndex(df_test['Date']).year
df_test['Month'] = pd.DatetimeIndex(df_test['Date']).month
df_test['Day'] = pd.DatetimeIndex(df_test['Date']).day

df_train.head()

,Sales,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,...,Open,Promo,StateHoliday,SchoolHoliday,store_encode,sales_storetype,Sales_date,Year,Month,Day
0,6267.0,253,a,c,250.0,NaN,NaN,1,5.0,2013.0,...,1.0,1.0,0,0.0,5908.065574,7425.648995,2013-07-19,2013,7,19
1,5900.0,653,d,c,7520.0,7.0,2014.0,1,45.0,2009.0,...,1.0,0.0,0,0.0,5854.905149,6843.009539,2013-03-01,2013,3,1
2,8468.0,676,b,b,1410.0,9.0,2008.0,0,NaN,NaN,...,1.0,1.0,0,0.0,7000.188544,8152.210169,2013-12-17,2013,12,17
3,5011.0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,...,1.0,1.0,0,0.0,4782.823684,6762.479180,2013-06-20,2013,6,20
4,5437.0,351,a,a,5290.0,11.0,2012.0,1,5.0,2013.0,...,1.0,0.0,0,0.0,5882.175141,6491.087624,2014-06-07,2014,6,7


In [12]:
def create_a_timeseries_dummy_from_weeks(df, var_week, var_year, date_new, var_dummy):
    #Create a date out of the Promo2 variables so that we can create a time-series variable for Promo2
    df['temp_date'] = df[var_year] * 1000 + df[var_week]  * 10 + 0
    df[date_new] = pd.to_datetime(df['temp_date'], format='%Y%W%w')
    ## Create an array to Group the data by group. and make dummy Store by Store
    for Store, grouped in df.groupby('Store'):
      if [df['Sales_date'] >= df[date_new]]:
        df[var_dummy] = 1
      else:
        df[var_dummy] = 0
    df = df.drop(columns=['temp_date'])
    df.head()

    
def create_a_timeseries_dummy_from_months(df, var_month, var_year, date_new, var_dummy):
    #Create a date out of the Competition variables so that we can create a time-series variable for Competitioin
    df['temp_date'] = df[var_year] * 1000 + df[var_month]  * 10 + 0
    ## Create an array to Group the data by group. and make dummy Store by Store
    df[date_new] = pd.to_datetime(df['temp_date'], format='%Y%m%w')
    for Store, grouped in df.groupby('Store'):
      if [df['Sales_date'] >= df[date_new]]:
        df[var_dummy] = 1
      else:
        df[var_dummy] = 0
    df = df.drop(columns=['temp_date'])    
    df.head()
    
    

In [13]:
create_a_timeseries_dummy_from_weeks(df=df_train        ,
                          var_week='Promo2SinceWeek'  ,
                          var_year='Promo2SinceYear'  ,
                          date_new='Promo2_start_date',
                          var_dummy='Promo2_yes'
                                    )

create_a_timeseries_dummy_from_weeks(df=df_test        ,
                          var_week='Promo2SinceWeek'  ,
                          var_year='Promo2SinceYear'  ,
                          date_new='Promo2_start_date',
                          var_dummy='Promo2_yes'
                                    )

In [14]:
create_a_timeseries_dummy_from_months(df=df_train                  ,
                          var_month='CompetitionOpenSinceMonth'  ,
                          var_year='CompetitionOpenSinceYear'   ,
                          date_new='Competition_start_date'    ,
                          var_dummy='Competition_yes'
                                    )

create_a_timeseries_dummy_from_months(df=df_test                 ,
                          var_month='CompetitionOpenSinceMonth'  ,
                          var_year='CompetitionOpenSinceYear'   ,
                          date_new='Competition_start_date'    ,
                          var_dummy='Competition_yes'
                                    )


In [15]:
df_train.columns

Index(['Sales', 'Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'Date', 'DayOfWeek', 'Open',
       'Promo', 'StateHoliday', 'SchoolHoliday', 'store_encode',
       'sales_storetype', 'Sales_date', 'Year', 'Month', 'Day', 'temp_date',
       'Promo2_start_date', 'Promo2_yes', 'Competition_start_date',
       'Competition_yes'],
      dtype='object')

In [16]:
#Drop columns for RandomForest Regressor

df_train = df_train.drop(columns=['Date','Store','Promo2','Promo2SinceWeek','Promo2SinceYear','CompetitionOpenSinceMonth','CompetitionOpenSinceYear'])

df_test = df_test.drop(columns=['Date','Promo2','Store','Promo2SinceWeek','Promo2SinceYear','CompetitionOpenSinceMonth','CompetitionOpenSinceYear'])
df_train.head()

,Sales,StoreType,Assortment,CompetitionDistance,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,store_encode,sales_storetype,Sales_date,Year,Month,Day,temp_date,Promo2_start_date,Promo2_yes,Competition_start_date,Competition_yes
0,6267.0,a,c,250.0,5.0,1.0,1.0,0,0.0,5908.065574,7425.648995,2013-07-19,2013,7,19,NaN,2013-02-10,1,NaT,1
1,5900.0,d,c,7520.0,1.0,1.0,0.0,0,0.0,5854.905149,6843.009539,2013-03-01,2013,3,1,2014070.0,2009-11-15,1,2014-07-01,1
2,8468.0,b,b,1410.0,2.0,1.0,1.0,0,0.0,7000.188544,8152.210169,2013-12-17,2013,12,17,2008090.0,NaT,1,2008-09-01,1
3,5011.0,c,a,1270.0,4.0,1.0,1.0,0,0.0,4782.823684,6762.479180,2013-06-20,2013,6,20,2008090.0,NaT,1,2008-09-01,1
4,5437.0,a,a,5290.0,6.0,1.0,0.0,0,0.0,5882.175141,6491.087624,2014-06-07,2014,6,7,2012110.0,2013-02-10,1,2012-11-01,1


In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397979 entries, 0 to 397978
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Sales                   397979 non-null  float64       
 1   StoreType               397979 non-null  object        
 2   Assortment              397979 non-null  object        
 3   CompetitionDistance     397979 non-null  float64       
 4   DayOfWeek               397979 non-null  float64       
 5   Open                    397979 non-null  float64       
 6   Promo                   397979 non-null  float64       
 7   StateHoliday            397979 non-null  string        
 8   SchoolHoliday           397979 non-null  float64       
 9   store_encode            397979 non-null  float64       
 10  sales_storetype         397979 non-null  float64       
 11  Sales_date              397979 non-null  datetime64[ns]
 12  Year                    397979

In [18]:
#Encoding-One Hot Enconder

import category_encoders as ce

# these are already dummies 'Open','Promo', 'SchoolHoliday'
ce_one = ce.OneHotEncoder(cols=['StateHoliday','StoreType','Assortment'])
df_train = ce_one.fit_transform(df_train)
df_test = ce_one.fit_transform(df_test)

In [19]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397979 entries, 0 to 397978
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Sales                   397979 non-null  float64       
 1   StoreType_1             397979 non-null  int64         
 2   StoreType_2             397979 non-null  int64         
 3   StoreType_3             397979 non-null  int64         
 4   StoreType_4             397979 non-null  int64         
 5   Assortment_1            397979 non-null  int64         
 6   Assortment_2            397979 non-null  int64         
 7   Assortment_3            397979 non-null  int64         
 8   CompetitionDistance     397979 non-null  float64       
 9   DayOfWeek               397979 non-null  float64       
 10  Open                    397979 non-null  float64       
 11  Promo                   397979 non-null  float64       
 12  StateHoliday_1          397979

In [20]:
#Drop columns for RandomForest Regressor

df_train = df_train.drop(columns=['Sales_date','Promo2_start_date','Competition_start_date','temp_date'])

df_test = df_test.drop(columns=['Sales_date','Promo2_start_date','Competition_start_date','temp_date'])
df_train.head()

,Sales,StoreType_1,StoreType_2,StoreType_3,StoreType_4,Assortment_1,Assortment_2,Assortment_3,CompetitionDistance,DayOfWeek,...,StateHoliday_3,StateHoliday_4,SchoolHoliday,store_encode,sales_storetype,Year,Month,Day,Promo2_yes,Competition_yes
0,6267.0,1,0,0,0,1,0,0,250.0,5.0,...,0,0,0.0,5908.065574,7425.648995,2013,7,19,1,1
1,5900.0,0,1,0,0,1,0,0,7520.0,1.0,...,0,0,0.0,5854.905149,6843.009539,2013,3,1,1,1
2,8468.0,0,0,1,0,0,1,0,1410.0,2.0,...,0,0,0.0,7000.188544,8152.210169,2013,12,17,1,1
3,5011.0,0,0,0,1,0,0,1,1270.0,4.0,...,0,0,0.0,4782.823684,6762.479180,2013,6,20,1,1
4,5437.0,1,0,0,0,0,0,1,5290.0,6.0,...,0,0,0.0,5882.175141,6491.087624,2014,6,7,1,1


In [21]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397979 entries, 0 to 397978
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Sales                397979 non-null  float64
 1   StoreType_1          397979 non-null  int64  
 2   StoreType_2          397979 non-null  int64  
 3   StoreType_3          397979 non-null  int64  
 4   StoreType_4          397979 non-null  int64  
 5   Assortment_1         397979 non-null  int64  
 6   Assortment_2         397979 non-null  int64  
 7   Assortment_3         397979 non-null  int64  
 8   CompetitionDistance  397979 non-null  float64
 9   DayOfWeek            397979 non-null  float64
 10  Open                 397979 non-null  float64
 11  Promo                397979 non-null  float64
 12  StateHoliday_1       397979 non-null  int64  
 13  StateHoliday_2       397979 non-null  int64  
 14  StateHoliday_3       397979 non-null  int64  
 15  StateHoliday_4   

In [22]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99397 entries, 0 to 99396
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Sales                99397 non-null  float64
 1   StoreType_1          99397 non-null  int64  
 2   StoreType_2          99397 non-null  int64  
 3   StoreType_3          99397 non-null  int64  
 4   StoreType_4          99397 non-null  int64  
 5   Assortment_1         99397 non-null  int64  
 6   Assortment_2         99397 non-null  int64  
 7   Assortment_3         99397 non-null  int64  
 8   CompetitionDistance  99397 non-null  float64
 9   DayOfWeek            99397 non-null  float64
 10  Open                 99397 non-null  float64
 11  Promo                99397 non-null  float64
 12  StateHoliday_1       99397 non-null  int64  
 13  StateHoliday_2       99397 non-null  int64  
 14  StateHoliday_3       99397 non-null  int64  
 15  StateHoliday_4       99397 non-null 

## 6. Split and Scale Traindataset

In [23]:
#Split train dataset

X_train = df_train.drop(columns=['Sales'])
y_train = np.asarray(df_train.loc[:,'Sales'])

X_test = df_test.drop(columns=['Sales'])
y_test = np.asarray(df_test.loc[:,'Sales'])


In [24]:
from sklearn.preprocessing import power_transform
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


In [25]:
#Scale 
scal = StandardScaler()
X_train_scaled = scal.fit_transform(X_train)

X_test_scaled = scal.fit_transform(X_test)

## 7.  Model Development ( training and prediction on training dataset)

### 7.1 Linear regression

In [26]:
from sklearn.linear_model import LinearRegression
#Model Linear Regression
regr = LinearRegression()

#Fittig on training Data
regr.fit(X_train_scaled, y_train)

#prediction
y_pred_train_linear = regr.predict(X_train_scaled)

In [27]:
#Building required metric

def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])


In [28]:
train_linear_score = metric(y_pred_train_linear,y_train)
train_linear_score

28.277525829253996

### 7.2 Random Forest

In [29]:
#Model Random Forest
from sklearn.ensemble import RandomForestRegressor

#Fittig on training Data
clf = RandomForestRegressor(n_estimators=300, n_jobs=2)
clf.fit(X_train_scaled, y_train)
#prediction
y_pred_train_random = clf.predict(X_train_scaled)


In [30]:
#Random Forest
train_RF_score = metric(y_pred_train_random,y_train)
train_RF_score

6.767801249369968

### 7.3 Gradient Boosted Trees

In [31]:
import xgboost as xgb
# XGBregressor 
xgbr = xgb.XGBRegressor(n_estimators = 1500, max_depth = 7, learning_rate = 0.15, n_jobs= -1)

#Fitting on training Data
xgbr.fit(X_train_scaled, y_train)

#predict with xgbr
y_pred_xgbr_train = xgbr.predict(X_train_scaled)


/home/jjrs93/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.36ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/jjrs93/.local/lib/python3.8/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [32]:
#XGBR_Train
metric_xgbr_train = metric(y_pred_xgbr_train, y_train)
metric_xgbr_train

12.87211382388552

## 8. Results using test datset



### 8.1 Linear regression

In [33]:
#Prediction

y_pred_test_linear = regr.predict(X_test_scaled)

#Metric

test_linear_score = metric(y_pred_test_linear,y_test)
test_linear_score

47888389997.07782

### 8.2 Random Forest

In [40]:
#Prediction
y_pred_test_random = clf.predict(X_test_scaled)

#Metric

test_RF_score = metric(y_pred_test_random,y_test)

print('The accuracy on the test data using Random Forest Regressor is {}'.format(test_RF_score))


The accuracy on the test data using Random Forest Regressor is 21.097603987019948


### 8.3 XGBR

In [38]:
#Prediction

y_pred_xgbr_test = xgbr.predict(X_test_scaled)


#Metric
metric_xgbr_test = metric(y_pred_xgbr_test, y_test)
metric_xgbr_test

print('The accuracy on the test data using XGBR is {}'.format(metric_xgbr_test))


The accuracy on the test data using XGBR is 18.972245359480944


## 9.Conclusion and further steps

1.The best accuracy has been reached after featuring engineering on the test dataset using XGRB : 18,972

2.Regarding the problem I would have used a different metric which allow me to keep using the samples with 0 sales.(for the excersise I have been asked to use RMSPE)

3.It would be possible to increase the XGBR accuracy by GridSearchCV and/or keep working on feature engineering